In [44]:
import os
import ast
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import keras
from keras import optimizers
from keras.utils import np_utils
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Input, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization
from keras.models import Sequential
from keras import backend as K

print(os.listdir("data"))

['test_simplified.csv', 'train_simplified.zip', 'train_simplified']


In [2]:
TRAIN_PATH = "data/train_simplified/"
TEST_PATH = "data/test_simplified.csv"
SUBMISSION_NAME = 'submission.csv'

In [3]:
train = pd.DataFrame()

for file in os.listdir(TRAIN_PATH):
    train = train.append(pd.read_csv(TRAIN_PATH + file, usecols = [1, 5], nrows = 100))
train = shuffle(train, random_state=123)  

In [4]:
len(train)

34000

In [5]:
# fixing label in the training set
train['word'] = train['word'].replace(' ', '_', regex=True)

# get labels and one-hot encode them.
n_classes = train['word'].unique()
labels = pd.get_dummies(train['word']).values
train.drop(['word'], axis=1, inplace=True)
print("Total number of labels: "+str(len(labels)))

Total number of labels: 34000


In [6]:
BASE_SIZE = 256
RE_SIZE = (70,70)

def datapoints_to_img(raw_strokes, size=256, lw=6, time_color=True):
    
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    return cv2.resize(img, RE_SIZE, interpolation=cv2.INTER_AREA)

In [7]:
#  training datapoints stacked in a list
drawings_train = [ast.literal_eval(pts) for pts in train['drawing'].values]
print(len(drawings_train))

34000


In [8]:
training_images = []
i = 0
for stroke in drawings_train:
    img = datapoints_to_img(stroke)
    training_images.append(img)
    print('processed {}/{}'.format(i+1,len(drawings_train)),end = '\r', flush=True)
    i+=1

In [9]:
training_images = np.array(training_images)

In [10]:
len(training_images)

34000

In [12]:
print(training_images.shape)

(34000, 70, 70)


In [14]:
x_train, x_val, y_train, y_val = train_test_split(training_images, labels, test_size=0.1, random_state=1)

In [34]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((30600, 70, 70, 1), (30600, 340), (3400, 70, 70), (3400, 340))

In [48]:
img_rows = 70
img_cols = 70

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_val = x_val.reshape(x_val.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
print(input_shape)

(70, 70, 1)


In [36]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255
x_val /= 255

In [37]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((30600, 70, 70, 1), (30600, 340), (3400, 70, 70, 1), (3400, 340))

In [38]:
# now feed it to the model

In [42]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(340, activation='softmax'))

In [45]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

In [46]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 68, 68, 32)        320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 66, 66, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 33, 33, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 33, 33, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 69696)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8921216   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
__________

In [50]:
epochs = 5
batch_size = 128

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,validation_data=(x_val, y_val))

Train on 30600 samples, validate on 3400 samples
Epoch 1/5
30600/30600 [==============================] - 591s 19ms/step - loss: 5.8300 - acc: 0.0020 - val_loss: 5.8305 - val_acc: 0.0015
Epoch 2/5
  256/30600 [..............................] - ETA: 9:24 - loss: 5.8294 - acc: 0.0039    